# Testing Envelopes on Data Samples

In [1]:
# Some useful modules for notebooks
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10, 3]
mpl.rcParams['figure.dpi'] = 300

from tqdne.conf import DATASETDIR
import numpy as np
from tqdne.conf import Config
config = Config()

In [3]:
from tqdne.dataset import WaveformDataset
from tqdne.representations import LogMaxEnvelope

path_train = config.datasetdir / Path(config.data_upsample_train)
print(path_train)
train_dataset = WaveformDataset(path_train, LogMaxEnvelope(config), reduced=1024)

/users/fmachado/data/japan/data_upsample_train.h5


In [4]:
s = train_dataset.waveform[0]
s.shape

(1, 1024)

In [5]:
train_dataset[0]

[[0.07638332]]


/users/fmachado/tqdne/tqdne/representations.py:39: RuntimeWarning: invalid value encountered in log
  return np.concatenate([np.log(envelope), scaled_signal], axis=-2)


{'high_res': tensor([[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [-0.0018, -0.0028,  0.0016,  ..., -0.0458, -0.0020,  0.0524]]),
 'cond': tensor([ 1.2616, -0.5696,  0.0859, -0.3657, -0.4295])}

In [6]:
e = np.max(np.abs(s), axis=-1, keepdims=True)
e = np.repeat(e, s.shape[-1], axis=-1)
sig = s / e
e = np.log10(e)

In [18]:
e

array([[-1.1170014, -1.1170014, -1.1170014, ..., -1.1170014, -1.1170014,
        -1.1170014]], dtype=float32)